## Calculating Boundary Conditions
Reynolds Number for external flow: 
$$ Re_{ext} = \dfrac{\rho U D}{\mu}$$
- $\rho$: Density of external flow
- $U$: Velocity of external flow
- $D$: Diameter of pipe
- $\mu$: viscosity of external flow

In [7]:
rho = 1.225 # [kg/m^3] of air 
u = 0.01   # [m/s], velocity of flow around pipe, ???unsure of this quantity
diameters = [0.00454,0.00394,0.00802]   # m, [0] inlet, [1] coil (coil d=0.3mm), [2] mouthpiece
mu = 1.802*10**(-5)  # kg/m*s, viscority of air

def reynoldsnum(u,d):
    re = (rho * u *d )/mu  # Reylnolds number
    return(re)

reynolds = []  # inlet, coils, mouthpiece
for diam in d:
    reynolds.append(reynoldsnum(u,diam))

# For analysis of external flow around the mouthpiece
d = diameters[2]
re = reynolds[2]
reynolds

[3.0862930077691453, 2.678412874583796, 5.451997780244173]

## Nusselt Number
The Nusselt number can be calculated from the Reynolds number and the Prandtl number. Given the Reynolds number we can calculate the Nusselt number from this empirical correlation: 
$$ Nu = c * Re^{m}*Pr^{1/3}$$
Note: $c$ & $m$ are constants (look up in book: https://hyominsite.files.wordpress.com/2015/03/fundamentals-of-heat-and-mass-transfer-6th-edition.pdf)

<img src="nusselt_empirical_correlation.png" width=50%>



In [8]:
pr = 0.71 # Prandtl number at room temperature
# Given Reynolds number = 67980.02219755827
if 0.4 < re < 4:
    c,m = 0.989,0.330    
if 4 < re < 40:
    c,m = 0.911,0.385    
if 40 < re < 4000:
    c,m = 0.683,0.466
if 4000 < re < 40000:
    c,m = 0.193,0.618
if 40000 < re < 400000:
    c,m = 0.027,0.805

In [9]:
nu = c * (re**(m))*(pr**(1/3))

## Heat Flux 
The nusselt number is a nondimensionalization of the heat flux 
$$ Nu = \dfrac{h_{ext}*d}{k_{air}} $$ 
we are solving for the heat flux: $h_{ext}$, given the Nusselt number we have: 
$$ h_{ext} = \dfrac{Nu*k_{air}}{d}$$
Note: $k_{air}$ is the kinematic viscosity of air. 

In [10]:
k_air = 0.024676  # kinematic viscosity of air

In [11]:
h_ext = nu*k_air/d

## Test: Choice of Cross Flow Velocity (U)

In [12]:
import pandas as pd

In [13]:
def heatflux(rho,u,d,mu,pr,k_air):
    re = (rho * u *d )/mu  # Reylnolds number
    # empirical correlations 
    if 0.4 < re < 4:
        c,m = 0.989,0.330    
    if 4 < re < 40:
        c,m = 0.911,0.385    
    if 40 < re < 4000:
        c,m = 0.683,0.466
    if 4000 < re < 40000:
        c,m = 0.193,0.618
    if 40000 < re < 400000:
        c,m = 0.027,0.805
    nu = c * (re**(m))*(pr**(1/3))
    h_ext = nu*k_air/d
    return(h_ext)

In [14]:
# Try a variety of external flow rates: 
u = [0.0009,0.001,0.005,0.01,0.05,0.1,0.49,0.5]
hf = []
for vel in u:
    h_ext = heatflux(rho,vel,d,mu,pr,k_air)
    hf.append(h_ext)

In [15]:
utest = pd.DataFrame({'Cross Flow (m/s)':u, 'Heat Flux':hf})
utest

,Cross Flow (m/s),Heat Flux
0,0.0009,2.146246
1,0.0010,2.222181
2,0.0050,3.779546
3,0.0100,4.804101
4,0.0500,8.927207
5,0.1000,12.083020
6,0.4900,25.340015
7,0.5000,25.579704


## Mesh Calculations

In [25]:
import numpy as np

In [26]:
L =  0.06203 # mm, includes all airflow (inside device and airflow)
# L = 0.00454 # mm diameter
u_int = 0.5 # m/s
re_int = rho*u_int*L/mu  # for internal flow
re_int

2108.40038845727

In [18]:
# c_f: skin friction coeffient - based on empirical result ????
c_f = ((2.0*np.log10(re_int))-6.5)**(-2.3) 
c_f

81.10198718950386

In [19]:
#based on a formula????
c_f = 1.328/np.sqrt(re_int)
c_f

0.028921547785850973

In [20]:
# wall shear stress 
tau_w = (0.5*rho*u_int*2)*c_f
tau_w

0.017714448018833723

In [21]:
# friction velocity
u_tau = np.sqrt(tau_w/rho)
u_tau

0.1202529579383621

In [23]:
# initial guess
# y_plus = rho*y_p*u_tau/mu
# y_p = y_plus*mu/(rho*u_tau)
# if in post processing y+ is too large then make smaller <1.0